In [ ]:
import math
import numpy as np

In [ ]:
%%bash -vv
git clone https://github.com/kpu/kenlm.git
mkdir kenlm/build
cd kenlm/build
cmake ..
make -j 4
pip3 install https://github.com/kpu/kenlm/archive/master.zip

-- The C compiler identification is GNU 7.5.0
-- The CXX compiler identification is GNU 7.5.0
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Could NOT find Eigen3 (missing: Eigen3_DIR)
-- Looking for pthread.h
-- Looking for pthread.h - found
-- Looking for pthread_create
-- Looking for pthread_create - not found
-- Looking for pthread_create in pthreads
-- Looking for pthread_create in pthreads - not found
-- Looking for pthread_create in pthread
-- Looking for pthread_create in pthread - found
-- Found Threads: TRUE  
-- Boost version: 

git clone https://github.com/kpu/kenlm.git
Cloning into 'kenlm'...
mkdir kenlm/build
cd kenlm/build
cmake ..
make -j 4
pip3 install https://github.com/kpu/kenlm/archive/master.zip


In [ ]:
!wget http://www.statmt.org/lm-benchmark/1-billion-word-language-modeling-benchmark-r13output.tar.gz

--2021-11-13 14:40:10--  http://www.statmt.org/lm-benchmark/1-billion-word-language-modeling-benchmark-r13output.tar.gz
Resolving www.statmt.org (www.statmt.org)... 129.215.197.184
Connecting to www.statmt.org (www.statmt.org)|129.215.197.184|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1792209805 (1.7G) [application/x-gzip]
Saving to: ‘1-billion-word-language-modeling-benchmark-r13output.tar.gz’

llion-word-language   0%[                    ]   7.54M  58.8KB/s    eta 8h 24m ^C


In [ ]:
!tar -zxvf 1-billion-word-language-modeling-benchmark-r13output.tar.gz

1-billion-word-language-modeling-benchmark-r13output/
1-billion-word-language-modeling-benchmark-r13output/training-monolingual.tokenized.shuffled/
1-billion-word-language-modeling-benchmark-r13output/training-monolingual.tokenized.shuffled/news.en-00024-of-00100

gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


In [ ]:
!cat 1-billion-word-language-modeling-benchmark-r13output/training-monolingual.tokenized.shuffled/* > 1b.txt

In [ ]:
!/content/kenlm/build/bin/lmplz -o 5 -S 90% -T ./ < 1b.txt > 1b.arpa

=== 1/5 Counting and sorting n-grams ===
Reading /content/1b.txt
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
tcmalloc: large alloc 4337344512 bytes == 0x55ff54a90000 @  0x7f0484fa21e7 0x55ff5204a7a2 0x55ff51fe551e 0x55ff51fc42eb 0x55ff51fb0066 0x7f048313bbf7 0x55ff51fb1baa
tcmalloc: large alloc 20240924672 bytes == 0x5600572fa000 @  0x7f0484fa21e7 0x55ff5204a7a2 0x55ff520397ca 0x55ff5203a208 0x55ff51fc4308 0x55ff51fb0066 0x7f048313bbf7 0x55ff51fb1baa
****************************************************************************************************
Unigram tokens 35370671 types 439259
=== 2/5 Calculating and sorting adjusted counts ===
Chain sizes: 1:5271108 2:2399819776 3:4499662336 4:7199459328 5:10499212288
tcmalloc: large alloc 10499219456 bytes == 0x55ff54a90000 @  0x7f0484fa21e7 0x55ff5204a7a2 0x55ff520397ca 0x55ff5203a208 0x55ff51fc48d7 0x55ff51fb0066 0x7f048313bbf7 0x55ff51fb1baa
tcmalloc: large alloc 2399821824 bytes ==

In [ ]:
!/content/kenlm/build/bin/build_binary 1b.arpa 1b.bin

Reading 1b.arpa
----5---10---15---20---25---30---35---40---45---50---55---60---65---70---75---80---85---90---95--100
****************************************************************************************************
SUCCESS


In [ ]:
!git clone https://github.com/chrisjbryant/lmgec-lite.git

Cloning into 'lmgec-lite'...
remote: Enumerating objects: 127, done.
remote: Total 127 (delta 0), reused 0 (delta 0), pack-reused 127
Receiving objects: 100% (127/127), 2.28 MiB | 15.88 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [ ]:
!python3 /content/lmgec-lite/lmgec.py </content/lmgec-lite/bible.txt> -mdl </content/1b.bin> -o <res.txt>


/bin/bash: -c: line 0: syntax error near unexpected token `newline'
/bin/bash: -c: line 0: `python3 /content/lmgec-lite/lmgec.py </content/lmgec-lite/bible.txt> -mdl </content/1b.bin> -o <res.txt>'


In [ ]:
import kenlm
model = kenlm.Model('1b.arpa')

In [ ]:
print('{0}-gram model'.format(model.order))

5-gram model


In [ ]:
sentence = 'The European developement of a forward-looking and comphrensive society is doh'.lower()
print(sentence)
print(model.score(sentence))

the european developement of a forward-looking and comphrensive society is doh
-51.663604736328125


In [ ]:
def score(s):
    return sum(prob for prob, _, _ in model.full_scores(s))

assert (abs(score(sentence) - model.score(sentence)) < 1e-3)

In [ ]:
# Show scores and n-gram matches
words = ['<s>'] + sentence.split() + ['</s>']
for i, (prob, length, oov) in enumerate(model.full_scores(sentence)):
    print('{0} {1}: {2}'.format(prob, length, ' '.join(words[i+2-length:i+2])))
    if oov:
        print('\t"{0}" is an OOV'.format(words[i+1]))

-3.681581497192383 2: <s> the
-5.012550354003906 2: the european
-7.147407531738281 1: developement
	"developement" is an OOV
-2.28902530670166 1: of
-1.8301318883895874 2: of a
-5.099890232086182 2: a forward-looking
-2.1487231254577637 2: forward-looking and
-7.749031066894531 1: comphrensive
	"comphrensive" is an OOV
-4.058502674102783 1: society
-1.485923171043396 2: society is
-7.842093467712402 1: doh
-3.318742513656616 1: </s>


In [ ]:
# Find out-of-vocabulary words
for w in words:
    if not w in model:
        print('"{0}" is an OOV'.format(w))

"developement" is an OOV
"comphrensive" is an OOV


In [ ]:
#Stateful query
state = kenlm.State()
state2 = kenlm.State()
#Use <s> as context.  If you don't want <s>, use model.NullContextWrite(state).
model.BeginSentenceWrite(state)
accum = 0.0
accum += model.BaseScore(state, "a", state2)
accum += model.BaseScore(state2, "sentence", state)
#score defaults to bos = True and eos = True.  Here we'll check without the end
#of sentence marker.  
assert (abs(accum - model.score("a sentence", eos = False)) < 1e-3)
accum += model.BaseScore(state, "</s>", state2)
assert (abs(accum - model.score("a sentence")) < 1e-3)

### beamsearch and kenlm

In [ ]:
!git clone --recursive https://github.com/parlance/ctcdecode.git
%cd ctcdecode

Cloning into 'ctcdecode'...
remote: Enumerating objects: 1102, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 1102 (delta 16), reused 28 (delta 13), pack-reused 1063
Receiving objects: 100% (1102/1102), 780.91 KiB | 15.94 MiB/s, done.
Resolving deltas: 100% (529/529), done.
Submodule 'third_party/ThreadPool' (https://github.com/progschj/ThreadPool.git) registered for path 'third_party/ThreadPool'
Submodule 'third_party/kenlm' (https://github.com/kpu/kenlm.git) registered for path 'third_party/kenlm'
Cloning into '/content/ctcdecode/third_party/ThreadPool'...
remote: Enumerating objects: 82, done.        
remote: Total 82 (delta 0), reused 0 (delta 0), pack-reused 82        
Cloning into '/content/ctcdecode/third_party/kenlm'...
remote: Enumerating objects: 14051, done.        
remote: Counting objects: 100% (364/364), done.        
remote: Compressing objects: 100% (296/296), done.        
remote: Total 14051 (delta 10

In [ ]:
!wget https://kaldi-asr.org/models/5/4gram_big.arpa.gz

--2021-11-13 09:26:03--  https://kaldi-asr.org/models/5/4gram_big.arpa.gz
Resolving kaldi-asr.org (kaldi-asr.org)... 46.101.158.64
Connecting to kaldi-asr.org (kaldi-asr.org)|46.101.158.64|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 99823907 (95M) [application/x-gzip]
Saving to: ‘4gram_big.arpa.gz’

4gram_big.arpa.gz   100%[===================>]  95.20M  20.7MB/s    in 5.6s    

2021-11-13 09:26:09 (17.1 MB/s) - ‘4gram_big.arpa.gz’ saved [99823907/99823907]



In [ ]:
!pip install .

Processing /content/ctcdecode
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for ctcdecode: filename=ctcdecode-1.0.3-cp37-cp37m-linux_x86_64.whl size=13266975 sha256=9254e97182d05d50ff95006f951a678e96ae1271b47cb2d212199c1d9c85b4d8
  Stored in directory: /tmp/pip-ephem-wheel-cache-wufxfjjm/wheels/da/bb/b4/233de9fd7927245208e27bcf688bf5680ae3f3874be2895eef
Successfully built ctcdecode


#KALDI

In [ ]:
%cd /content

/content


In [ ]:
!make

mkdir -p include lib bin
make init
make[1]: Entering directory '/content/srilm'
for subdir in misc dstruct lm flm lattice utils zlib; do \
	(cd $subdir/src; make SRILM=/content/srilm MACHINE_TYPE=i686-m64 OPTION= MAKE_PIC= init) || exit 1; \
done
make[2]: Entering directory '/content/srilm/misc/src'
cd ..; /content/srilm/sbin/make-standard-directories
make ../obj/i686-m64/STAMP ../bin/i686-m64/STAMP
make[3]: Entering directory '/content/srilm/misc/src'
mkdir ../obj/i686-m64/
touch ../obj/i686-m64/STAMP
mkdir ../bin/i686-m64/
touch ../bin/i686-m64/STAMP
make[3]: Leaving directory '/content/srilm/misc/src'
make[2]: Leaving directory '/content/srilm/misc/src'
make[2]: Entering directory '/content/srilm/dstruct/src'
cd ..; /content/srilm/sbin/make-standard-directories
make ../obj/i686-m64/STAMP ../bin/i686-m64/STAMP
make[3]: Entering directory '/content/srilm/dstruct/src'
mkdir ../obj/i686-m64/
touch ../obj/i686-m64/STAMP
mkdir ../bin/i686-m64/
touch ../bin/i686-m64/STAMP
make[3]: Leaving 

In [ ]:
!unzip srilm-1.7.3.zip

Archive:  srilm-1.7.3.zip
   creating: srilm-1.7.3/
  inflating: srilm-1.7.3/Makefile    
  inflating: srilm-1.7.3/ACKNOWLEDGEMENTS  
  inflating: srilm-1.7.3/CHANGES     
  inflating: srilm-1.7.3/README      
  inflating: srilm-1.7.3/License     
  inflating: srilm-1.7.3/Copyright   
  inflating: srilm-1.7.3/INSTALL     
  inflating: srilm-1.7.3/RELEASE     
   creating: srilm-1.7.3/visual_studio/
   creating: srilm-1.7.3/visual_studio/vs2005/
  inflating: srilm-1.7.3/visual_studio/vs2005/go.msbuild  
  inflating: srilm-1.7.3/visual_studio/vs2005/README.txt  
  inflating: srilm-1.7.3/visual_studio/vs2005/srilm.sln  
   creating: srilm-1.7.3/visual_studio/vs2005/hidden-ngram/
  inflating: srilm-1.7.3/visual_studio/vs2005/hidden-ngram/hidden-ngram.vcproj  
   creating: srilm-1.7.3/visual_studio/vs2005/nbest-pron-score/
  inflating: srilm-1.7.3/visual_studio/vs2005/nbest-pron-score/nbest-pron-score.vcproj  
   creating: srilm-1.7.3/visual_studio/vs2005/segment-nbest/
  inflating: srilm-1

In [ ]:
!srilm/bin/i686-m64/ngram-count -order 4 -wbdiscount -write-vocab librispeech-vocab.txt -text 1b_out.txt -lm lm.arpa

In [ ]:
!iconv -f cp1251 -t utf8 1b.txt -o 1b_out.txt

iconv: illegal input sequence at position 18308082


In [ ]:
%%bash
srilm/bin/i686-m64/ngram -lm lm.arpa -prune 1e-8 -write-lm lm_tglarge.arpa.gz
srilm/bin/i686-m64/ngram -lm lm.arpa -prune 1e-7 -write-lm lm_tgmed.arpa.gz
srilm/bin/i686-m64/ngram -lm lm.arpa -prune 3e-7 -write-lm lm_tgsmall.arpa.gz

In [ ]:
!pwd

/content


In [ ]:
%cd extras

/content/kaldi/tools/extras


In [ ]:
!./nstall_srilm.sh

/bin/bash: install_srilm.sh: command not found
